In [182]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from tensorboardX import SummaryWriter
import tqdm
import os
import uuid
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

In [183]:
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

unique_id = str(uuid.uuid4())

plot = plt.figure()

def weights_initialize(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('relu'))
        module.bias.data.fill_(0.01)
        
class _TransModel(nn.Module):
    """ Model for DQN """

    def __init__(self, input_len, output_len):
        super(_TransModel, self).__init__()
        
        self.fc1 = nn.Sequential(
            torch.nn.Linear(input_len, 1024),
            torch.nn.BatchNorm1d(1024),
            nn.ReLU()
        )
        self.fc1.apply(weights_initialize)
        
        self.fc2 = nn.Sequential(
            torch.nn.Linear(1024, 256),
            # torch.nn.BatchNorm1d(128),
            nn.ReLU()
        )
        self.fc2.apply(weights_initialize)
        
        self.output_layer = nn.Sequential(
            torch.nn.Linear(256, output_len)
        )
        self.output_layer.apply(weights_initialize)
        
    def forward(self, input):
        x = self.fc1(input)
        x = self.fc2(x)
        
        return self.output_layer(x)

class TransModel():
    def __init__(self, input_len, ouput_len, learning_rate = 0.0001):
        self.model = _TransModel(input_len, ouput_len)
        
        if use_cuda:
            print("Using GPU")
            self.model = self.model.cuda()
        else:
            print("Using CPU")
        self.steps = 0
        # self.model = nn.DataParallel(self.model)
        self.optimizer = Adam(self.model.parameters(), lr = learning_rate)
        self.loss_fn = nn.MSELoss(reduction='mean')
        
        self.steps = 0
        
    def predict(self, input, steps, learning):
        
        output = self.model(input).squeeze(1)
        #reward, next_state = output[0], output[1:]

        return output

    def predict_batch(self, input):
        output = self.model(input)
        #reward, next_state = output[:, 0], output[:, 1:]
        return output

    def fit(self, state, target_state):
        loss = self.loss_fn(state, target_state)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.steps += 1
        return loss
    
    def save(self):
        file_path = '../models_mb/nexus-HP-transition-model-predict-4.pt'
        torch.save(self.model.state_dict(), file_path)

<Figure size 432x288 with 0 Axes>

In [184]:
def groundTruth(hp):
    if (hp >= 0 and hp <= 250):
        category = 0
    
    if (hp >= 251 and hp <= 500):
        category = 1

    if (hp >= 501 and hp <= 750):
        category = 2
        
    if (hp >= 751 and hp <= 1000):
        category = 3
    
    if (hp >= 1001 and hp <= 1250):
        category = 4

    if (hp >= 1251 and hp <= 1500):
        category = 5
    
    if (hp >= 1501 and hp <= 1750):
        category = 6

    if (hp >= 1751 and hp <= 2000):
        category = 7
    return category

In [185]:
data = torch.load('data/test_dataset.pt')
l = len(data)

for i in range(0, len(data)):
    data[i][1] = [ data[i][1][63] / 2000 , data[i][1][64] / 2000 , data[i][1][65] / 2000 , data[i][1][66] / 2000 ]
    
    data[i][0][1:4] = np.true_divide( data[i][0][1:4], 30) # Normalize P1 top buildings
    data[i][0][4:7] = np.true_divide( data[i][0][4:7], 30) # Normalize P1 bottom buildings

    data[i][0][8:11] = np.true_divide( data[i][0][8:11], 30) # Normalize P2 top buildings
    data[i][0][11:14] = np.true_divide( data[i][0][11:14], 30) # Normalize P2 bottom buildings
    
    data[i][0][63] = data[i][0][63] / 2000 # Normalize P1 Top Nexus HP
    data[i][0][64] = data[i][0][64] / 2000 # Normalize P2 Top Nexus HP
    
    data[i][0][65] = data[i][0][65] / 2000 # Normalize P1 Bottom Nexus HP
    data[i][0][66] = data[i][0][66] / 2000 # Normalize P2 Bottom Nexus HP

    data[i][0][0] = data[i][0][10] / 1500 # Normalize P1 Minerals
    
data = np.array(data)
print(data[0][0].shape)


(68,)


In [186]:
tmodel = TransModel(len(data[0][0]), len(data[0][1]))
tmodel.model.load_state_dict(torch.load("./nexus-HP-transition-model-predict-4.pt"))

Using GPU


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [187]:

a = torch.from_numpy(np.stack(data[:, 0])).type(FloatTensor)
b = torch.from_numpy(np.stack(data[0:2, 0])).type(FloatTensor)
print(a.shape)
print(b.shape)

print("A: ", tmodel.model(a))
print("B: ", tmodel.model(b))


torch.Size([11090, 68])
torch.Size([2, 68])
A:  tensor([[0.4687, 0.6796, 0.5010, 0.7168],
        [0.7387, 0.9872, 0.5975, 0.4174],
        [0.8627, 1.0003, 0.9994, 0.9008],
        ...,
        [0.9944, 0.0102, 0.6156, 1.0006],
        [0.2592, 0.9929, 0.9168, 0.9497],
        [0.9610, 0.9995, 0.9994, 0.9193]], device='cuda:0',
       grad_fn=<AddmmBackward>)
B:  tensor([[0.5187, 0.5989, 0.6647, 0.9921],
        [0.9729, 1.0018, 0.9783, 0.6108]], device='cuda:0',
       grad_fn=<AddmmBackward>)
